In [1]:
import ee
import os
import pandas as pd
import numpy as np
import yaml
import math
import matplotlib.pyplot as plt

ee.Initialize()

In [2]:
#pulls in text from FIA csv and generates relevant features
csvname='CA_PLOT.csv'
treename='CA_TREE.csv'
fia=pd.read_csv(csvname,usecols=('PLOT','COUNTYCD','LAT','LON','INVYR','PLOT_STATUS_CD'),delimiter=',', dtype=float)
trees=pd.read_csv(treename, delimiter=',', usecols=('PLOT','COUNTYCD','SPCD','STATUSCD'),dtype="int64")

In [3]:
#defines tree class for starting at top row of fia (PLOT.csv), also defining emptyspace as nan
#0's mean no match between trees and plot csv (i.e. not sampled, plot_status=2,3)
#returns array with class for each plot in order of PLOT.csv file
def define_tree_class(num) :
    i=0
    subfia=fia['PLOT']
    subtrees=trees['PLOT']
    speciesarraycolumn=[]
    deathcodecolumn=[]
    counter=0
    while i < num :
        try:
            statusarray=(
                trees['STATUSCD'][np.where(int(subfia[i])==subtrees)[0]]
                )
            statusmajority=np.argmax(np.bincount(statusarray))
            numdead=float(len(statusarray[statusarray==2]))
            numtotal=float(len(statusarray))
            deathprop=float(numdead/numtotal)
            if deathprop > .8 :
                deathcode=1
            if deathprop < .7999999 :
                deathcode=0
            if statusmajority == 3 :
                deathcode=2
                
        except ValueError :
                deathcode=0
                counter=counter+1
        try:
            speciesarray=(
                trees['SPCD'][np.where(int(subfia[i])==subtrees)[0]]
                )
            speciescode= np.argmax(np.bincount(speciesarray))
            speciesunique= np.unique(speciesarray)
            if len(np.where(speciesarray==speciescode)[0])/len(speciesarray) > .8 :
                speciescode=1001
            else :
                speciescode=speciescode
        except ValueError:

            speciescode=0
        speciesarraycolumn=np.append(speciesarraycolumn,speciescode)
        deathcodecolumn=np.append(deathcodecolumn,deathcode)
        i=i+1
    finalclasses=[]
    i=0
    for number in speciesarraycolumn :
        finaldeathcode=deathcodecolumn[i]
        if finaldeathcode == 0 :
            if 0 < number < 265:
                plotclass=1
            if number > 264:
                plotclass=2
            if number < 1:
                plotclass=0
            if number > 1000:
                plotclass=3
        if finaldeathcode == 1 :
            if 0 < number < 265:
                plotclass=4
            if number > 264:
                plotclass=5
            if number < 1:
                plotclass=0
            if number > 1000:
                plotclass=6
        if finaldeathcode == 2 :
            plotclass=0
        finalclasses=np.append(finalclasses,plotclass)
        i=i+1
    return(finalclasses)
    #arrays below can be used to match ids with plotstatus codes to check functionality
    #array=np.zeros((num,2))
    #array[:,0]=speciesarraycolumn
    #array[:,1]=fia['PLOT_STATUS_CD'][0:100]
    #print(array)
        
finalclasses=define_tree_class(50)
print(finalclasses[finalclasses>3])


[]


In [4]:
#builds features from top of csv file for designated number of lines
#returns tuple with ee.featurecollection sized according to num in index 0 
#number of features collected is in position 1
def build_features(num) :
        i=0
        listsamples=[]
        while i < num :
            lonlat=[fia['LON'][i],fia['LAT'][i]]
            thiseventyear= int(fia['INVYR'][i])
            if finalclasses[i] > 0:
                singlepoint= ee.Feature(ee.Geometry.Point(lonlat),
                        {
                          "system:index": str(len(listsamples)),
                          "plot_class": finalclasses[i],
                          "label": "A",
                          "event_year": thiseventyear
                        })
                listsamples=np.append(listsamples,singlepoint)
            if finalclasses[i] < 1 :
                chance=float(np.random.rand(1))
                if chance > .25 :
                    singlepoint= ee.Feature(ee.Geometry.Point(lonlat),
                        {
                          "system:index": str(len(listsamples)),
                          "plot_class": finalclasses[i],
                          "label": "A",
                          "event_year": thiseventyear
                        })
                    listsamples=np.append(listsamples,singlepoint)
                if chance < 24.9999999:
                    listsamples=listsamples
            i=i+1
        samples=ee.FeatureCollection(list(listsamples))
        return(samples,len(listsamples))
output=build_features(3)
samples=output[0]
numberoffeatures=output[1]
    

In [43]:
naip = ee.ImageCollection('USDA/NAIP/DOQQ').select(['R', 'G', 'B'])


time_stamp = 'system:time_start'  # image acquisition year
radius = 55  # pixels

def get_neighbs(feature):
    feature_geom = feature.geometry()
    feature_naip = naip.filterBounds(feature_geom)
    feat_img_year = feature.get('event_year')
    search_start = ee.Date.fromYMD(feat_img_year, 1, 1).advance(-1.5, 'year')
    search_stop = ee.Date.fromYMD(feat_img_year, 12, 31).advance(1.5, 'year')
    this_date_range = ee.DateRange(search_start, search_stop)
    feature_naip = feature_naip.filterDate(this_date_range)
    def get_this_date_image_list(date): 
      this_date_naip = feature_naip.filter(ee.Filter.eq(time_stamp, date))
      neighbs = this_date_naip.mean().int().neighborhoodToBands(ee.Kernel.square(radius))
      neighb_dict = neighbs.reduceRegion(geometry=feature_geom,reducer=ee.Reducer.first(),scale=1)
      full_feature = ee.Feature(feature_geom, neighb_dict)
      first_image = ee.Image(this_date_naip.first())
      feat_dict = {'_SAMPLE_ID': feature.get('system:index'), 
                   '_LABEL': feature.get('label'), 
                   '_CLASS': feature.get('plot_class'),
                   '_IMG_DATE_IS_MATCH': ee.Date(date).get('year').eq(feat_img_year),
                   '_IMG_DATE': ee.Date(date).format('MMM d, y'), 
                   '_IMG_ID': first_image.get('system:index'),
                   '_IMG_RES': first_image.projection().nominalScale()
                   }
      return full_feature.set(feat_dict)
    image_dates = ee.List(feature_naip.distinct(time_stamp).aggregate_array(time_stamp))
    image_dates_image_list = image_dates.map(get_this_date_image_list)
    full_features = ee.FeatureCollection(image_dates_image_list)
    return full_features.filter(ee.Filter.neq('R_0_0', None))


sample_reds = samples.map(get_neighbs)
export_task = ee.batch.Export.table.toDrive(sample_reds.flatten(), folder='FIA_Training')
export_task.start()

Define functions used to develop tabular data for samples into images that can be inspected during the model building process

In [5]:
def range_scale(x):
    """Range scale a 1D array."""
    return (x-min(x))/(max(x)-min(x))

def aggregate_ee_reductions_data(csvs_dir, patterns):
    """Append data for each sample into one large data frame."""
    every_df = pd.DataFrame()
    for file_name in os.listdir(csvs_dir):
        matched = all(fnmatch.fnmatch(file_name, p) for p in patterns)
        if matched:
            file_size = os.path.getsize(os.path.join(csvs_dir, file_name))
            if file_size != 1: 
                this_df = pd.read_csv(os.path.join(csvs_dir, file_name))
                every_df = every_df.append(this_df)
    return every_df.dropna().reset_index(drop=True)

def get_index_of_sorted_bands_and_pixels(table, band_map):
    """Retrieve indices of the sorted columns (combinations of bands and pixel locations)."""
    col_names = table.columns
    channel = [i.split('_')[0] for i in col_names]
    x_pix_loc = [i.split('_')[1] for i in col_names]
    y_pix_loc = [i.split('_')[2] for i in col_names]
    df = pd.DataFrame({'channel':channel, 'x_pix_loc':x_pix_loc, 'y_pix_loc':y_pix_loc})
    df = df.replace({'channel': band_map}).convert_objects(convert_numeric=True)
    df_sorted = df.sort(['channel', 'y_pix_loc', 'x_pix_loc'], ascending=[True, True, True])
    return df_sorted.index

def save_tabular_image_data_to_png(table, image_size, bands, pngs_dir,
                                   metadata_columns, band_columns, 
                                   band_map={'R': 0, 'G': 1, 'B': 2, 'QA':3},
                                   groups=['training', 'test'], prediction=False):
    """Loop through each row (sample) in the table and write the data to disk as a PNG."""
    metadata = table.iloc[:, metadata_columns]
    unsorted_pixel_data = table.iloc[:, band_columns]
    sort_indices = get_index_of_sorted_bands_and_pixels(unsorted_pixel_data, band_map)
    pixel_data = unsorted_pixel_data.iloc[:, sort_indices] 
    num_image_files = pixel_data.shape[0]
    image_group = np.random.choice(2, num_image_files, p=[0.9, 0.1])
    row_n = image_size * image_size * bands
    dataset = np.ndarray(shape=(num_image_files, image_size, image_size, bands),
                         dtype=np.uint8)
    for image in range(num_image_files):
        image_vals = pixel_data.iloc[image, :]
        dataset[image, :, :, band_map['R']] = np.reshape(np.array([image_vals[:row_n/bands]]), 
                       (image_size, image_size))
        dataset[image, :, :, band_map['G']] = np.reshape(np.array([image_vals[row_n/bands:row_n/bands*2]]), 
                       (image_size, image_size))
        dataset[image, :, :, band_map['B']] = np.reshape(np.array([image_vals[row_n/bands*2:row_n/bands*3]]), 
                       (image_size, image_size))
        image_coords = yaml.load(metadata.ix[image, '.geo'])['coordinates']
        #coords_string = '(' + str(image_coords[1]) + ', ' + str(image_coords[0]) + ')'
        imageclass=str(metadata['_CLASS'][int(image)])
        if prediction:
            png_name = 'record ' + str(image) + ' ' + imageclass + '.png'
            group_dir = 'fia_unknown'
            image_dir = os.path.join(pngs_dir, group_dir)
        else:
            image_year = metadata.ix[image, '_IMG_DATE']
            record_id = str(int(metadata.ix[image, '_SAMPLE_ID']))
            png_name = 'record ' + record_id + ' ' + image_year + ' ' + imageclass + '.png'
            group_dir = groups[image_group[image]]
            image_dir = os.path.join(pngs_dir, group_dir, metadata.ix[image, '_LABEL'])
        if not os.path.exists(image_dir):
            os.makedirs(image_dir)
#        print(image_dir)
#       print(os.path.exists(image_dir))
        plt.imsave(os.path.join(image_dir, png_name), dataset[image, :, :, 0:bands])
    

In [6]:
ee_reds = pd.read_csv('sample_context_55_pixel_radius.csv')

print(ee_reds.shape)

(114, 36972)


In [8]:
col_names = ee_reds.columns
radius=int(55)
print(col_names)
metadata_cols = ['system:index', '.geo', '_LABEL', '_SAMPLE_ID',
                 '_IMG_DATE', '_IMG_DATE_IS_MATCH', '_IMG_ID', '_IMG_RES','_CLASS']
metadata_indices = [col_names.get_loc(x) for x in metadata_cols]
print(metadata_indices)
band_indices = [x for x in range(ee_reds.shape[1]) if x not in metadata_indices]
print(math.sqrt(len(band_indices)/3))

save_tabular_image_data_to_png(ee_reds, image_size=radius*2+1, bands=3, 
                               pngs_dir='sample_pngs',
                               metadata_columns=metadata_indices , band_columns=band_indices)

Index([u'system:index', u'B_-10_-1', u'B_-10_-10', u'B_-10_-11', u'B_-10_-12',
       u'B_-10_-13', u'B_-10_-14', u'B_-10_-15', u'B_-10_-16', u'B_-10_-17',
       ...
       u'R_9_8', u'R_9_9', u'_CLASS', u'_IMG_DATE', u'_IMG_DATE_IS_MATCH',
       u'_IMG_ID', u'_IMG_RES', u'_LABEL', u'_SAMPLE_ID', u'.geo'],
      dtype='object', length=36972)
[0, 36971, 36969, 36970, 36965, 36966, 36967, 36968, 36964]
111.0


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:24: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:25: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [30]:
x=ee_reds.columns.get_values()

'R_9_8'